In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 49.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=5ddf8696135e6484247f5ad6bc026e5f0dd9892dfa2b5ad998ec0a3153e282d8
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('test_1').getOrCreate()

In [5]:
df = spark.read.csv('/content/drive/MyDrive/BDAI Project/unbalaced_20_80_dataset.csv',inferSchema=True, header=True)

In [6]:
df.show()

+-------+--------------------+--------------+--------+-------------+--------+--------+--------------------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+------------------+---------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------+------------+-----------+------------------+------------------+-----------+-----------+-----------+--------------------+-------------------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+-------------------+-------------------+-----------+-----------+------------------+------------------+------------------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+------------------+----------------+------------------+--------------+--------------+----------------+

In [7]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Flow ID: string (nullable = true)
 |-- Src IP: string (nullable = true)
 |-- Src Port: integer (nullable = true)
 |-- Dst IP: string (nullable = true)
 |-- Dst Port: integer (nullable = true)
 |-- Protocol: integer (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Flow Duration: integer (nullable = true)
 |-- Tot Fwd Pkts: integer (nullable = true)
 |-- Tot Bwd Pkts: integer (nullable = true)
 |-- TotLen Fwd Pkts: double (nullable = true)
 |-- TotLen Bwd Pkts: double (nullable = true)
 |-- Fwd Pkt Len Max: double (nullable = true)
 |-- Fwd Pkt Len Min: double (nullable = true)
 |-- Fwd Pkt Len Mean: double (nullable = true)
 |-- Fwd Pkt Len Std: double (nullable = true)
 |-- Bwd Pkt Len Max: double (nullable = true)
 |-- Bwd Pkt Len Min: double (nullable = true)
 |-- Bwd Pkt Len Mean: double (nullable = true)
 |-- Bwd Pkt Len Std: double (nullable = true)
 |-- Flow Byts/s: double (nullable = true)
 |-- Flow Pkts/s: double (nul

In [8]:
df.groupBy('Label').count().show()

+------+-------+
| Label|  count|
+------+-------+
|  ddos|1294529|
|Benign|6321980|
+------+-------+



### Balance:
Benign: 83% - DDOS: 17%

In [9]:
from pyspark.sql.functions import col,isnan, when, count

In [10]:
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+---+-------+------+--------+------+--------+--------+---------+-------------+------------+------------+---------------+---------------+---------------+---------------+----------------+---------------+---------------+---------------+----------------+---------------+-----------+-----------+-------------+------------+------------+------------+-----------+------------+-----------+-----------+-----------+-----------+------------+-----------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+----------+----------+-----------+-----------+------------+-----------+-----------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+------------+----------------+----------------+--------------+--------------+----------------+--------------+--------------+----------------+----------------+----------------+----------------+----------------+-----------------+-----------------+-

### NO NULLS

In [11]:
df_ddos = df.filter(df['Label']=='ddos')
df_benign = df.filter(df['Label']=='Benign')

In [12]:
oversample_ratio = df_benign.count()/df_ddos.count()

In [13]:
df_ddos_oversampled = df_ddos.sample(withReplacement=True, fraction=oversample_ratio-1, seed=1)

In [14]:
df_final = df_benign.unionAll(df_ddos_oversampled)

In [15]:
df_final.show()

+-------+--------------------+---------------+--------+---------------+--------+--------+--------------------+-------------+------------+------------+---------------+---------------+---------------+---------------+------------------+------------------+---------------+---------------+------------------+------------------+--------------------+-------------------+------------------+------------------+------------+------------+------------+------------------+------------------+-----------+-----------+------------+------------------+------------------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+-------------------+-------------------+-----------+-----------+------------------+------------------+------------------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+------------------+------------------+------------------+--------------+--------------+---------

In [16]:
df_final.groupBy('Label').count().show()

+------+-------+
| Label|  count|
+------+-------+
|Benign|6321980|
|  ddos|5029280|
+------+-------+



### Balance: 
Benign: 56% - DDOS: 44% 

In [17]:
df_final.count()

11351260

In [18]:
my_cols_df_full = df_final.columns

In [19]:
my_cols_df_full

['_c0',
 'Flow ID',
 'Src IP',
 'Src Port',
 'Dst IP',
 'Dst Port',
 'Protocol',
 'Timestamp',
 'Flow Duration',
 'Tot Fwd Pkts',
 'Tot Bwd Pkts',
 'TotLen Fwd Pkts',
 'TotLen Bwd Pkts',
 'Fwd Pkt Len Max',
 'Fwd Pkt Len Min',
 'Fwd Pkt Len Mean',
 'Fwd Pkt Len Std',
 'Bwd Pkt Len Max',
 'Bwd Pkt Len Min',
 'Bwd Pkt Len Mean',
 'Bwd Pkt Len Std',
 'Flow Byts/s',
 'Flow Pkts/s',
 'Flow IAT Mean',
 'Flow IAT Std',
 'Flow IAT Max',
 'Flow IAT Min',
 'Fwd IAT Tot',
 'Fwd IAT Mean',
 'Fwd IAT Std',
 'Fwd IAT Max',
 'Fwd IAT Min',
 'Bwd IAT Tot',
 'Bwd IAT Mean',
 'Bwd IAT Std',
 'Bwd IAT Max',
 'Bwd IAT Min',
 'Fwd PSH Flags',
 'Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'Fwd Header Len',
 'Bwd Header Len',
 'Fwd Pkts/s',
 'Bwd Pkts/s',
 'Pkt Len Min',
 'Pkt Len Max',
 'Pkt Len Mean',
 'Pkt Len Std',
 'Pkt Len Var',
 'FIN Flag Cnt',
 'SYN Flag Cnt',
 'RST Flag Cnt',
 'PSH Flag Cnt',
 'ACK Flag Cnt',
 'URG Flag Cnt',
 'CWE Flag Count',
 'ECE Flag Cnt',
 'Down/Up Ratio',
 'Pkt Size 

In [20]:
df_final.show()

+-------+--------------------+---------------+--------+---------------+--------+--------+--------------------+-------------+------------+------------+---------------+---------------+---------------+---------------+------------------+------------------+---------------+---------------+------------------+------------------+--------------------+-------------------+------------------+------------------+------------+------------+------------+------------------+------------------+-----------+-----------+------------+------------------+------------------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+-------------------+-------------------+-----------+-----------+------------------+------------------+------------------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+------------------+------------------+------------------+--------------+--------------+---------

In [21]:
from pyspark.sql.functions import dayofmonth, hour, dayofyear, month, year, weekofyear, format_number, date_format

In [22]:
df_final.select(df_final['Timestamp']).show()

+--------------------+
|           Timestamp|
+--------------------+
| 20/02/2018 10:36:27|
|04/07/2017 12:15:...|
| 20/02/2018 08:55:58|
| 20/02/2018 11:53:46|
| 20/02/2018 10:06:32|
| 20/02/2018 11:28:42|
| 20/02/2018 12:13:16|
| 20/02/2018 10:16:50|
| 20/02/2018 04:22:35|
| 20/02/2018 08:56:55|
| 20/02/2018 10:36:01|
|03/07/2017 06:39:...|
| 20/02/2018 05:10:09|
| 20/02/2018 02:42:50|
|03/07/2017 08:01:...|
| 20/02/2018 10:38:11|
| 20/02/2018 10:01:49|
|04/07/2017 12:25:...|
| 20/02/2018 11:17:24|
| 20/02/2018 02:54:06|
+--------------------+
only showing top 20 rows



In [23]:
df_final.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Flow ID: string (nullable = true)
 |-- Src IP: string (nullable = true)
 |-- Src Port: integer (nullable = true)
 |-- Dst IP: string (nullable = true)
 |-- Dst Port: integer (nullable = true)
 |-- Protocol: integer (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Flow Duration: integer (nullable = true)
 |-- Tot Fwd Pkts: integer (nullable = true)
 |-- Tot Bwd Pkts: integer (nullable = true)
 |-- TotLen Fwd Pkts: double (nullable = true)
 |-- TotLen Bwd Pkts: double (nullable = true)
 |-- Fwd Pkt Len Max: double (nullable = true)
 |-- Fwd Pkt Len Min: double (nullable = true)
 |-- Fwd Pkt Len Mean: double (nullable = true)
 |-- Fwd Pkt Len Std: double (nullable = true)
 |-- Bwd Pkt Len Max: double (nullable = true)
 |-- Bwd Pkt Len Min: double (nullable = true)
 |-- Bwd Pkt Len Mean: double (nullable = true)
 |-- Bwd Pkt Len Std: double (nullable = true)
 |-- Flow Byts/s: double (nullable = true)
 |-- Flow Pkts/s: double (nul

In [24]:
str_columnList = [item[0] for item in df_final.dtypes if item[1].startswith('string')]

In [25]:
str_columnList

['Flow ID', 'Src IP', 'Dst IP', 'Timestamp', 'Label']

In [26]:
num_cols = []
for i in df_final.columns:
  if i not in str_columnList:
    num_cols.append(i)

In [30]:
num_cols.append('Label')

In [31]:
df_nums = df_final.select(num_cols)

In [32]:
df_nums.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Src Port: integer (nullable = true)
 |-- Dst Port: integer (nullable = true)
 |-- Protocol: integer (nullable = true)
 |-- Flow Duration: integer (nullable = true)
 |-- Tot Fwd Pkts: integer (nullable = true)
 |-- Tot Bwd Pkts: integer (nullable = true)
 |-- TotLen Fwd Pkts: double (nullable = true)
 |-- TotLen Bwd Pkts: double (nullable = true)
 |-- Fwd Pkt Len Max: double (nullable = true)
 |-- Fwd Pkt Len Min: double (nullable = true)
 |-- Fwd Pkt Len Mean: double (nullable = true)
 |-- Fwd Pkt Len Std: double (nullable = true)
 |-- Bwd Pkt Len Max: double (nullable = true)
 |-- Bwd Pkt Len Min: double (nullable = true)
 |-- Bwd Pkt Len Mean: double (nullable = true)
 |-- Bwd Pkt Len Std: double (nullable = true)
 |-- Flow Byts/s: double (nullable = true)
 |-- Flow Pkts/s: double (nullable = true)
 |-- Flow IAT Mean: double (nullable = true)
 |-- Flow IAT Std: double (nullable = true)
 |-- Flow IAT Max: double (nullable = true)
 |-- Flow

### Building the ML Model

In [33]:
df_nums.show()

+-------+--------+--------+--------+-------------+------------+------------+---------------+---------------+---------------+---------------+------------------+------------------+---------------+---------------+------------------+------------------+--------------------+-------------------+------------------+------------------+------------+------------+------------+------------------+------------------+-----------+-----------+------------+------------------+------------------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+-------------------+-------------------+-----------+-----------+------------------+------------------+------------------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+------------------+------------------+------------------+--------------+--------------+----------------+--------------+--------------+----------------+----------------+--

In [54]:
from pyspark.ml.feature import VectorAssembler

In [72]:
#df_nums.columns

In [55]:
assembler = VectorAssembler(inputCols=df_nums.columns[1:-1], outputCol='features')

In [56]:
output = assembler.transform(df_nums)

In [57]:
from pyspark.ml.feature import StringIndexer

In [58]:
indexer = StringIndexer(inputCol='Label',outputCol='LabelIndex')

In [59]:
output_fixed = indexer.fit(output).transform(output)

In [60]:
output_fixed.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Src Port: integer (nullable = true)
 |-- Dst Port: integer (nullable = true)
 |-- Protocol: integer (nullable = true)
 |-- Flow Duration: integer (nullable = true)
 |-- Tot Fwd Pkts: integer (nullable = true)
 |-- Tot Bwd Pkts: integer (nullable = true)
 |-- TotLen Fwd Pkts: double (nullable = true)
 |-- TotLen Bwd Pkts: double (nullable = true)
 |-- Fwd Pkt Len Max: double (nullable = true)
 |-- Fwd Pkt Len Min: double (nullable = true)
 |-- Fwd Pkt Len Mean: double (nullable = true)
 |-- Fwd Pkt Len Std: double (nullable = true)
 |-- Bwd Pkt Len Max: double (nullable = true)
 |-- Bwd Pkt Len Min: double (nullable = true)
 |-- Bwd Pkt Len Mean: double (nullable = true)
 |-- Bwd Pkt Len Std: double (nullable = true)
 |-- Flow Byts/s: double (nullable = true)
 |-- Flow Pkts/s: double (nullable = true)
 |-- Flow IAT Mean: double (nullable = true)
 |-- Flow IAT Std: double (nullable = true)
 |-- Flow IAT Max: double (nullable = true)
 |-- Flow

In [61]:
output_fixed.show()

+-------+--------+--------+--------+-------------+------------+------------+---------------+---------------+---------------+---------------+------------------+------------------+---------------+---------------+------------------+------------------+--------------------+-------------------+------------------+------------------+------------+------------+------------+------------------+------------------+-----------+-----------+------------+------------------+------------------+-----------+-----------+-------------+-------------+-------------+-------------+--------------+--------------+-------------------+-------------------+-----------+-----------+------------------+------------------+------------------+------------+------------+------------+------------+------------+------------+--------------+------------+-------------+------------------+------------------+------------------+--------------+--------------+----------------+--------------+--------------+----------------+----------------+--

In [62]:
final_data = output_fixed.select(['features','LabelIndex'])

In [65]:
final_data.show()

+--------------------+----------+
|            features|LabelIndex|
+--------------------+----------+
|(79,[0,1,2,3,4,5,...|       0.0|
|(79,[0,1,2,3,4,5,...|       0.0|
|[56889.0,3389.0,6...|       0.0|
|[51263.0,443.0,6....|       0.0|
|(79,[0,1,2,3,4,5,...|       0.0|
|(79,[0,1,2,3,4,5,...|       0.0|
|(79,[0,1,2,3,4,5,...|       0.0|
|(79,[0,1,2,3,4,17...|       0.0|
|(79,[0,1,2,3,4,5,...|       0.0|
|(79,[0,1,2,3,4,5,...|       0.0|
|(79,[0,1,2,3,4,5,...|       0.0|
|(79,[0,1,2,3,4,5,...|       0.0|
|(79,[0,1,2,3,4,17...|       0.0|
|[57775.0,3389.0,6...|       0.0|
|(79,[0,1,2,3,4,5,...|       0.0|
|(79,[0,1,2,3,4,5,...|       0.0|
|[49411.0,80.0,6.0...|       0.0|
|(79,[0,1,2,3,4,5,...|       0.0|
|(79,[0,1,2,3,4,5,...|       0.0|
|[52095.0,80.0,6.0...|       0.0|
+--------------------+----------+
only showing top 20 rows



In [66]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

### Using Tree Based models and Logistic regression

In [48]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier, LogisticRegression

In [68]:
dt = DecisionTreeClassifier(labelCol='LabelIndex',featuresCol='features')
rf = RandomForestClassifier(numTrees=100,labelCol='LabelIndex',featuresCol='features')
gbt = GBTClassifier(labelCol='LabelIndex',featuresCol='features')

In [73]:
#dt_model = dt.fit(train_data)
#rf_model = rf.fit(df_train)
#gb_model = gbt.fit(df_train)